In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pwd

/content


In [ ]:
workspace = '/content/drive/MyDrive/Courses/CS410/MP4/'

In [ ]:
import os
os.path.exists(workspace)

True

# Creating Base BM25 Candidate using Pyserini

First. We will create a candidate BM25 candidate ranking using the Pyserini library. This library is a python wrapper on Anserini which is a wrapper on lucene. First we format data in the format Pyserini expects to produce our collection file. 

In [ ]:
import json
doc_id = -1
with open('data/documents.txt','r') as f:
    with open('data/collection.jsonl','w') as w:
        for l in f:
            l = l.strip()
            doc_id += 1
            data = {"id": str(doc_id), "contents": l}
            w.write(json.dumps(data))
            w.write('\n')

Once we have the data in our intended format we go ahead and create an index

In [ ]:
! python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input data/ \
  --index indexes/410 \
  --generator DefaultLuceneDocumentGenerator \
  --threads 1 \
  --storePositions --storeDocvectors --storeRaw

Now that we have our index we go ahead and retrieve making a TREC formated ranking.

In [ ]:
from pyserini.search.lucene import LuceneSearcher
searcher = LuceneSearcher('indexes/410')
qid = 1
with open("data/queries.txt",'r') as f:
    with open('bm25-top1000.trec','w') as w:
        for l in f:
            query = l.strip()
            hits = searcher.search(query,k=1000)
            for i in range(len(hits)):
                w.write(f'{qid:5}\t{0}\t{hits[i].docid:4}\t{i+1:2}\t{hits[i].score:.5f}\tbm25\n')
            qid += 1